In [7]:
from math import floor, log2
from sympy import prime

In [1]:
# wiener vulnerable
m = 198614235373674103788888306985643587194108045477674049828293333354607555709
c = 60278311525973438866348054125035969957458310299959975267975269326631807541076198821617577977630397205355473918704575171192579993287663207864988439328835231524516323450595645371973555696071228527541776679612089193355470623559313376895794531673334509318279167607306947896618949387489794595845072549080046122231
d = 23009757856071844268568055912545458118138742425651340843696073482971286971615
e = 60622565810056765588076682990247867931467974177173929430876799924141527547159343311956425490167737740240046629439818049884054674496400421837121395839395671068760025748314749504507293154243658839233494420713870324550776601235709721477057208088323328590472437615906346795333144247497340280153973078100120070771
n = 76881499195179758719860576028129204256330463041538761885870902922146174453818918859116923311186372704033550511757618765150988598753482039061299751106213131549269527605251067558555225306491166622692816670944375066132653303087101319120517392175340003297430678476513934572673535247748275637537396895236079423941
p = 7763338934602697279748332648086085192267317357016587041443338139740907849594395036920888068586614237923957000946534416713286157525947261857424168392169367
q = 9903148612062279695184974394942430814302817287340215681696223192481788630483586667090617147331535537896655460738209367624045422088081722029580391041035523
assert d<(1/3)*(n**(1/4)), "d not small enough for Wiener's attack"

In [2]:

def temp_gcd(a, b):
    while a != b: 
        if a > b:
            a = a - b
        else:
            b = b - a
    return a
def gcd(a, b): # b > a
    q = []
    q_0 = 0
    a_0 = a
    b_0 = b
    ### extended gcd ###
    s_0 = 1
    s_1 = 0
    t_0 = 0
    t_1 = 1
    ### extended gcd ###
    while True:
        q_0 = 0
        while b_0>=a_0: #  and a_0!=0
            b_0 -= a_0
            q_0+=1
        q.append(q_0)
        ### extended gcd ###
        temp_s = s_1
        s_1 = s_0-q_0*s_1
        s_0 = temp_s
        temp_t = t_1
        t_1 = t_0-q_0*t_1
        t_0 = temp_t
        # print(t_0, t_1, s_0, s_1)
        ### extended gcd ###
        if (q_0==0 or b_0==0): break
        rem = b_0
        b_0 = a_0
        a_0 = rem
    return a_0, t_0, s_0
def lcm(a,b):
    return floor(abs(a*b)/gcd(a,b)[0])
ascii32bit = "abcdefghijklmnopqrstuvwxyz.,!?' "
def ascii32bit_encode(s: str):
    acc = 0
    i = 0
    for c in s:
        acc+=(ascii32bit.index(c) if c in ascii32bit else 31)<<i
        i+=5
    return acc
# edge case where last char has leading zeros, need to adjust bit shift offset accordingly
def ascii32bit_decode(s: int):
    acc = s
    ans = ""
    n = round(log2(s)) - 5
    n = n+(5-n%5) if n%5!=0 else n
    for i in range(n, -1, -5):
        temp = acc >> i
        temp2 = temp << i
        ans=ascii32bit[temp]+ans
        acc -= temp2
    return ans
# no longer lazy :) https://en.wikipedia.org/wiki/Modular_exponentiation
def expmod(b,e,m):
    c = 1
    for _ in range(0, e):
        c = b*c % m
    return c
def fast_mod_exp(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m
import random
# ported from https://stackoverflow.com/questions/2921406/calculate-primes-p-and-q-from-private-exponent-d-public-exponent-e-and-the
# given d e and n, find p and q
def pq_from_den(d, e, n):
    k = d*e-1
    if (k%2==0):
        r = k
        t = 0
        while (r%2==0):
            r = r >> 1
            t+=1
        success = False
        y = -1
        for i in range(1,101):
            g = -1
            g = random.randrange(0,n)
            y = fast_mod_exp(g,r,n)
            if y==1 or y==n-1:
                continue
            for j in range(1, t):
                x = fast_mod_exp(y,2,n)
                if x==1:
                    success = True
                    break
                if x==n-1:
                    continue
                y = x
            x = fast_mod_exp(y, 2, n)
            if (x==1):
                success=True
                break
        if (success):
            p = gcd(y-1, n)[0]
            q = n/p
            return p, q
        else:
            assert False, "not found"
    else:
        assert False, "not found"

In [3]:
# from sympy import lcm as lcm_sympy
# declare 
p = 9903148612062279695184974394942430814302817287340215681696223192481788630483586667090617147331535537896655460738209367624045422088081722029580391041035523 # prime(18000)
q = 7763338934602697279748332648086085192267317357016587041443338139740907849594395036920888068586614237923957000946534416713286157525947261857424168392169367 # prime(10000)
# assert q<p and p<2*q, f"{q}<{p}<{2*q} failed, is condition for wiener but not applicable here for some reason?"
# p*q (p and q are coprime)
n = 76881499195179758719860576028129204256330463041538761885870902922146174453818918859116923311186372704033550511757618765150988598753482039061299751106213131549269527605251067558555225306491166622692816670944375066132653303087101319120517392175340003297430678476513934572673535247748275637537396895236079423941

# https://en.wikipedia.org/wiki/Carmichael_function
# lambda_n of +ve n where a^m congruent 1 mod n for all a coprime to n
# implemented lcm from wikipedia too https://en.wikipedia.org/wiki/Euclidean_algorithm
lambda_n = lcm(q-1,p-1)
e =  60622565810056765588076682990247867931467974177173929430876799924141527547159343311956425490167737740240046629439818049884054674496400421837121395839395671068760025748314749504507293154243658839233494420713870324550776601235709721477057208088323328590472437615906346795333144247497340280153973078100120070771 #prime(2598) # 1<e<lambda_n and gcd(e, lambda_n)=1
# assert 1<e and e<lambda_n, f"e={e}, lambda_n={lambda_n}"
# assert gcd(e, lambda_n)[0]==1, f"e={e}, lambda_n={lambda_n}"
# a congruent b % m
# de congruent 1 mod n
# https://en.wikipedia.org/wiki/B%C3%A9zout%27s_identity
# https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm
_, s, t = gcd(e, lambda_n)
d = s if s>=0 else (-s*(lambda_n-1))
# p q n e d lambda_n

public_key = (n, e)
private_key = (d)
other_private = (p, q, lambda_n)

# M = "hi"
# m = ascii32bit_encode(M) # not padded for simplicity
# print(m)
# c = fast_mod_exp(m,e,n)
# d_m = fast_mod_exp(c,d,n)
# print(d_m)
# d_M = ascii32bit_decode(d_m)
# d_M
c = 60278311525973438866348054125035969957458310299959975267975269326631807541076198821617577977630397205355473918704575171192579993287663207864988439328835231524516323450595645371973555696071228527541776679612089193355470623559313376895794531673334509318279167607306947896618949387489794595845072549080046122231
d_m = fast_mod_exp(c,d,n)
print(d_m)
d_M = bytearray.fromhex(hex(d_m)[2:])
d_M

60278311525973438866348054125035969957458310299959975267975269326631807541076198821617577977630397205355473918704575171192579993287663207864988439328835231524516323450595645371973555696071228527541776679612089193355470623559313376895794531673334509318279167607306947896618949387489794595845072549080046122231


bytearray(b'U\xd6\xd3\x83a\t\x9d\xa8\x1f8 \xaaQ\xc8\xab\x84\xddL\x01y\x18\xf5\x8a\xe9Zc\x0b\xaf\xb1\x11\x8b\xea\xb48\x07i\x98g\xe7\xb5.UY\x03\xdc\xa7\x98WZ\x18\x10x>\xf2\x0b\xf4\x9f\xfa\x14jF\x976C \xab\x02J\x9fE\xdd1\xd8\xea\x8b6R\xc6\xb1}\xd8,\x85\x9f\x91\x9b\tG\xd0\x95\x12\x84\x1eHc&\xc5\xcf\x8f\xee\xf9\x1d2%\xa2\xb6\xd8\xe6\xa4\xc0 q\xb8#\xac\xed\x9f\x13\x94,\x8a\x8f\xcc\x125\xe24\xf7')

In [8]:
from sympy import lcm as lcm_sympy
from sympy import gcd as gcd_sympy
print(lcm(p-1,q-1))
print(lcm_sympy(p-1,q-1))
print(gcd(p-1,q-1))
print(gcd_sympy(p-1,q-1))
print(temp_gcd(p-1,q-1))

7763338934602697610279991366075870890202786738410789140159681174226801337352058608907171951766261108424433026594421028152351160631143283688348945366384640
38440749597589879359930288014064602128165231520769380942935451461073087226909459429558461655593186352016775255878809382575494299376741019530649875553106556941390990470137046312624091138987580026279086157070825963285660540195310620569406690335062042573827428932026124914444598958084330804276754945338323109526
(9903148612062279695184974394942430814302817287340215681696223192481788630483586667090617147331535537896655460738209367624045422088081722029580391041035522, 1, 0)
2
2


In [120]:
gcd(e, lambda_n)

(60622565810056765588076682990247867931467974177173929430876799924141527547159343311956425490167737740240046629439818049884054674496400421837121395839395671068760025748314749504507293154243658839233494420713870324550776601235709721477057208088323328590472437615906346795333144247497340280153973078100120070771,
 1,
 0)

In [ ]:
23009757856071844268568055912545458118138742425651340843696073482971286971615

In [116]:
fast_mod_exp(c, d, n)

198614235373674103788888306985643587194108045477674049828293333354607555709

In [102]:
d = 23009757856071844268568055912545458118138742425651340843696073482971286971615
e = 60622565810056765588076682990247867931467974177173929430876799924141527547159343311956425490167737740240046629439818049884054674496400421837121395839395671068760025748314749504507293154243658839233494420713870324550776601235709721477057208088323328590472437615906346795333144247497340280153973078100120070771
n = 76881499195179758719860576028129204256330463041538761885870902922146174453818918859116923311186372704033550511757618765150988598753482039061299751106213131549269527605251067558555225306491166622692816670944375066132653303087101319120517392175340003297430678476513934572673535247748275637537396895236079423941
c = 60278311525973438866348054125035969957458310299959975267975269326631807541076198821617577977630397205355473918704575171192579993287663207864988439328835231524516323450595645371973555696071228527541776679612089193355470623559313376895794531673334509318279167607306947896618949387489794595845072549080046122231
p = 7763338934602697279748332648086085192267317357016587041443338139740907849594395036920888068586614237923957000946534416713286157525947261857424168392169367
q = 9903148612062279695184974394942430814302817287340215681696223192481788630483586667090617147331535537896655460738209367624045422088081722029580391041035523
pq_from_den(d,e,n)

(7763338934602697279748332648086085192267317357016587041443338139740907849594395036920888068586614237923957000946534416713286157525947261857424168392169367,
 9.903148612062279e+153)

In [101]:
7763338934602697279748332648086085192267317357016587041443338139740907849594395036920888068586614237923957000946534416713286157525947261857424168392169367*q-n

0

In [100]:
(n//q).as_integer_ratio()

(7763338934602697279748332648086085192267317357016587041443338139740907849594395036920888068586614237923957000946534416713286157525947261857424168392169367,
 1)

In [83]:
7763338934602697610279991366075870890202786738410789140159681174226801337352058608907171951766261108424433026594421028152351160631143283688348945366384640*q

76881499195179761993164713303832860874957946358522233973497702959270880944976726732442195097580875062208365015947546038592951404097799956882728625874478335302777583937071338158039206130233935126035357020122410386773007109906173193946500141632378856493521195165460196550092392554418688014143887202260321566720

In [103]:
fast_mod_exp(c, d, n)

198614235373674103788888306985643587194108045477674049828293333354607555709

In [104]:
fast_mod_exp(198614235373674103788888306985643587194108045477674049828293333354607555709, e, n)

60278311525973438866348054125035969957458310299959975267975269326631807541076198821617577977630397205355473918704575171192579993287663207864988439328835231524516323450595645371973555696071228527541776679612089193355470623559313376895794531673334509318279167607306947896618949387489794595845072549080046122231

In [106]:
fast_mod_exp(
    60278311525973438866348054125035969957458310299959975267975269326631807541076198821617577977630397205355473918704575171192579993287663207864988439328835231524516323450595645371973555696071228527541776679612089193355470623559313376895794531673334509318279167607306947896618949387489794595845072549080046122231
    , d, p*q)

198614235373674103788888306985643587194108045477674049828293333354607555709

In [145]:
d, n, 1/3*pow(n, 0.25)

(545667645, 20964965407, 126.83884621086305)

In [25]:
e=60622565810056765588076682990247867931467974177173929430876799924141527547159343311956425490167737740240046629439818049884054674496400421837121395839395671068760025748314749504507293154243658839233494420713870324550776601235709721477057208088323328590472437615906346795333144247497340280153973078100120070771
n=76881499195179758719860576028129204256330463041538761885870902922146174453818918859116923311186372704033550511757618765150988598753482039061299751106213131549269527605251067558555225306491166622692816670944375066132653303087101319120517392175340003297430678476513934572673535247748275637537396895236079423941
c=60278311525973438866348054125035969957458310299959975267975269326631807541076198821617577977630397205355473918704575171192579993287663207864988439328835231524516323450595645371973555696071228527541776679612089193355470623559313376895794531673334509318279167607306947896618949387489794595845072549080046122231
d = '???'
m = '???'
# chal: implement wiener's attack from just wikipedia pages references :)
# first implement for example small rsa values for easy